In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Specify the paths to the parent folder containing "train" and "test"
parent_data_dir = "/content/drive/My Drive/finaldataset/skin"
train_data_dir = parent_data_dir + "/train"
test_data_dir = parent_data_dir + "/test"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)  # Normalize pixel values for test data

# Set the batch size
batch_size = 32

# Create training and testing data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)


Found 7452 images belonging to 6 classes.
Found 1644 images belonging to 6 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2


# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),# Pooling layer
    Dropout(0.5),
    layers.Dense(128, activation='relu'),  # Additional dense layer (adjust as needed)
    layers.Dense(6, activation='softmax')  # 23 disease classes + 1 for normal
])


from tensorflow.keras.optimizers import Adam

# Change the learning rate to a different value, e.g., 0.001
optimizer = Adam(learning_rate=0.001)

# Compile the model with the updated optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()  # View the model architecture


9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                           

In [ ]:
# Set the number of training epochs
epochs = 25
# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)


Epoch 1/25
233/233 [==============================] - 2232s 9s/step - loss: 0.9966 - accuracy: 0.5734 - val_loss: 10.5820 - val_accuracy: 0.1235
Epoch 2/25
233/233 [==============================] - 143s 609ms/step - loss: 0.8170 - accuracy: 0.6559 - val_loss: 3.5934 - val_accuracy: 0.3114
Epoch 3/25
233/233 [==============================] - 141s 606ms/step - loss: 0.7620 - accuracy: 0.6875 - val_loss: 2.4616 - val_accuracy: 0.4696
Epoch 4/25
233/233 [==============================] - 140s 602ms/step - loss: 0.6957 - accuracy: 0.7142 - val_loss: 3.4912 - val_accuracy: 0.3899
Epoch 5/25
233/233 [==============================] - 141s 607ms/step - loss: 0.6707 - accuracy: 0.7304 - val_loss: 3.8212 - val_accuracy: 0.3668
Epoch 6/25
233/233 [==============================] - 139s 598ms/step - loss: 0.6376 - accuracy: 0.7363 - val_loss: 4.4848 - val_accuracy: 0.2038
Epoch 7/25
233/233 [==============================] - 140s 600ms/step - loss: 0.6249 - accuracy: 0.7528 - val_loss: 7.0042 - 

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

52/52 [==============================] - 8s 156ms/step - loss: 1.9636 - accuracy: 0.6253
Test accuracy: 0.6253041625022888


In [17]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Load the trained model (assuming it's already loaded or defined in your Colab session)
# If not, you can load it using the code you provided earlier
# model = ...

# Upload an image file
uploaded = files.upload()

# Get the uploaded image file name (assuming only one file is uploaded)
uploaded_image_filename = list(uploaded.keys())[0]

# Define the path to the parent folder containing class labels (training data)
parent_data_dir = "/content/drive/My Drive/finaldataset/skin/train"

# Get the class labels from the subdirectories in the training data folder
class_labels =  ["Acne and Rosacea Photos","Eczema Photos","Normal","Psoriasis pictures Lichen Planus and related diseases","Tinea Ringworm Candidiasis and other Fungal Infections","vitiligo"]
# print(class_labels)
# Load and preprocess the uploaded image
img = image.load_img(uploaded_image_filename, target_size=(224, 224))  # Resize to (224, 224)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize pixel values

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the predicted class label using the extracted class labels
predicted_class_label = class_labels[predicted_class_index]

# Get the probability of the predicted class
predicted_probability = predictions[0][predicted_class_index]

# Display the results
print(f'Predicted class: {predicted_class_label}')

print(f'Probability: {predicted_probability:.2f}')



Saving 13candidaTongue082205.jpg to 13candidaTongue082205.jpg
1/1 [==============================] - 0s 23ms/step
Predicted class: Tinea Ringworm Candidiasis and other Fungal Infections
Probability: 0.43


In [ ]:
model.save('final_model_62.h5')
model.save('my_model.keras')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
import tensorflow as tf

# Load the saved model
loaded_model = tf.keras.models.load_model("final_model_62.h5")

parent_data_dir = "/content/drive/My Drive/finaldataset/skin/test/Normal/0_0_aidai_0029.jpg"

# Get the class labels from the subdirectories in the training data folder
class_labels =  ["Acne and Rosacea Photos","Eczema Photos","Normal","Psoriasis pictures Lichen Planus and related diseases","Tinea Ringworm Candidiasis and other Fungal Infections","vitiligo"]
# print(class_labels)
# Load and preprocess the uploaded image
img = image.load_img(uploaded_image_filename, target_size=(224, 224))  # Resize to (224, 224)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize pixel values

# Make predictions
predictions = loaded_model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the predicted class label using the extracted class labels
predicted_class_label = class_labels[predicted_class_index]

print(f'Predicted class: {predicted_class_label}')


NameError: ignored

In [9]:
from google.colab import files

# Download the file to your local system
files.download('final_model_62.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>